In [ ]:
# LSTM 15 YEar data without sentiment 
df_btc = pd.read_csv('/notebooks/BITCOIN_DATA.csv', parse_dates=['datetime'])
df_btc.sort_values('datetime', inplace=True)
df_btc = df_btc[['datetime', 'close','volume']]



train_df = df_btc[(df_btc['datetime'] >= '2011-01-01') & (df_btc['datetime'] <= '2023-12-31')].copy()
test_df = df_btc[df_btc['datetime'] > '2023-12-31'].copy()

# We use two features: 'close' and 'volume'
# To simplify the inverse transformation for the target ('close'),
# we use separate scalers for 'close' and 'volume'.
close_scaler = MinMaxScaler()
volume_scaler = MinMaxScaler()

# Fit scalers on training data and transform both training and test data
train_df['close_scaled'] = close_scaler.fit_transform(train_df[['close']])
train_df['volume_scaled'] = volume_scaler.fit_transform(train_df[['volume']])
test_df['close_scaled'] = close_scaler.transform(test_df[['close']])
test_df['volume_scaled'] = volume_scaler.transform(test_df[['volume']])

# Create a NumPy array combining the two features (order matters: [close, volume])
train_scaled = train_df[['close_scaled', 'volume_scaled']].values
test_scaled = test_df[['close_scaled', 'volume_scaled']].values

# ----------------------------
# 2. Creating Sequences for Time Series
# ----------------------------

# Define the lookback period (number of past days to use as input)
lookback = 30  # e.g., use the previous 30 days to predict the next day's close

def create_sequences(data, lookback):
    X, y = [], []
    for i in range(lookback, len(data)):
        # Use the previous 'lookback' days (each with 2 features) as input
        X.append(data[i - lookback:i])
        # The target is the close price (first column) at the current time step
        y.append(data[i, 0])
    return np.array(X), np.array(y)

# Create sequences for training and testing
X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ----------------------------
# 3. Model Building and Training
# ----------------------------

# Build a simple LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(lookback, 2)))
model.add(LSTM(50))
model.add(Dense(1))  # Predict a single value: the close price
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

# ----------------------------
# 4. Prediction and Inverse Transformation
# ----------------------------

# Generate predictions on the test set
predictions_scaled = model.predict(X_test)

# Inverse transform predictions (we only predicted the close price)
prediction_no_sentiment = close_scaler.inverse_transform(predictions_scaled)

# Also inverse transform the actual test targets for comparison
y_test_actual_no_sentiment = close_scaler.inverse_transform(y_test.reshape(-1, 1))

# Since we lose the first 'lookback' days when creating sequences,
# align the dates from test_df accordingly.
test_dates = test_df['datetime'].values[lookback:]


# ----------------------------
# 5. Visualization: Predicted vs Actual Close Prices
# ----------------------------

plt.figure(figsize=(12, 6))
plt.plot(test_dates, y_test_actual_no_sentiment, label='Actual Close Price', color='blue')
plt.plot(test_dates, prediction_no_sentiment, label='Predicted Close Price', color='red', linestyle='--')
plt.xlabel('Date')
plt.ylabel('BTC Close Price')
plt.title('BTC Close Price Prediction: Actual vs Predicted (Mar 15 - May 1, 2019)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
